![ups logo](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png/640px-Logo_Universidad_Polit%C3%A9cnica_Salesiana_del_Ecuador.png)

# Aprendizaje Automático P64

# Practica 02


## Redes Neuronales Problemas Multiclase y Optimización (fine tunning)

## Fase 2: Optimización de una red neuronal para clasificación multiclase con imágenes 

## Autor: Diego Tapia


### Importación de librerias

In [1]:

import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 

from time import time
from datetime import datetime, timedelta

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder, MinMaxScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
import pickle
import seaborn as sns
import copy

from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout

import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.models import model_from_json
from keras.models import load_model

from sklearn import metrics 
from sklearn.metrics import confusion_matrix  
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

from sklearn import set_config   

print("Módulos y clases importados")

2024-04-29 18:43:57.509098: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 18:43:57.513384: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 18:43:57.572959: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 18:43:59.173829: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Módulos y clases importados


### Carga del dataset

Se carga el dataset CIFAR10 de la libreria de keras

In [3]:
from tensorflow.keras.datasets import cifar10
cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

### Transformación de los datos

Se transforman las imágenes para ser usadas en una red neuronal clásica 

In [4]:
# Normalizamos tanto X_train como X_test, en este caso de forma sencilla porque los valores de los pixeles van de 0 a 255.
x_train = X_train/255
x_test = X_test/255
print("Normalización lista!")

#Ejemplo para verificar
print('x:',X_test[9999,1,1,1])
print("""x':""",x_test[9999,1,1,1])

Normalización lista!
x: 89
x': 0.34901960784313724


In [5]:
#Para transformar X en estructura clásica de ML (2D) para aplicar técnicas como SVM, Redes Neuronales Densas, KNN, etc.
X_trainReshapeImageRow=x_train.reshape((x_train.shape[0],3072))
print(X_trainReshapeImageRow.shape)
X_testReshapeImageRow=x_test.reshape((x_test.shape[0],3072))
print(X_testReshapeImageRow.shape)

(50000, 3072)
(10000, 3072)


Se unen los conjuntos de train y test para usar en la función GridSearchCV

In [10]:
# Concatenar conjunto entrenamiento y prueba en una sola variable "Y"
Y = np.concatenate((y_train, y_test), axis=0)
# Se carga en Xt los conjuntos de entrenamiento y testing convertidos
Xt = np.concatenate((X_trainReshapeImageRow, X_testReshapeImageRow), axis=0)

## Optimización

### 1.Parámetros de compilación

En este paso se ajustará los valores de: batch_size, epochs y optimizer. El objetivo es encontrar la mejor combinación entre estos parámetros.

In [2]:
#Función para convertir el tiempo de segundos al formato de días, horas, minutos y segundos.
def GetTime(gs_time):
    sec=timedelta(seconds=gs_time)
    d = datetime(1,1,1) + sec
    tiempoTotal=("%d días: %d horas: %d min: %d seg" % (d.day-1, d.hour, d.minute, d.second))
    return tiempoTotal

In [6]:
# Creacion del pipeline (opcional)

pipe = Pipeline(
    steps=[]
)

In [7]:
def create_model():


  model = Sequential([
      Dense(32, input_dim=X_trainReshapeImageRow.shape[1], activation='relu'),
      Dense(10, activation='softmax')
  ])
  #compilamos el modelo
  model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

  return model

Se crea listas que contienen los valores de los parámetros que se busca optimizar 

In [9]:
batch_size = [32,64,128,256,512,1024]
epochs = [10,25,50,75,100,150] 
optimizer = ['RMSprop','Adam']


#parámetros que queremos probar, y sus valores 
parameters = {'modelNN__batch_size': batch_size,
             'modelNN__epochs': epochs,
             'modelNN__optimizer': optimizer}

estimator = KerasClassifier(model=create_model, verbose=0)

pipe.steps.append(['modelNN',estimator])

print(pipe)

Pipeline(steps=[['modelNN',
                 KerasClassifier(model=<function create_model at 0x7a33961a6700>, verbose=0)]])


Se empieza a buscar la mejor combinación de parámetros con la función ```GridSearchCV```

In [ ]:
# se toma el tiempo inicial para calcular el tiempo de ejecución 
tic = time()

grid_search = GridSearchCV(estimator=pipe, param_grid=parameters, scoring='accuracy', cv=5, n_jobs=6, error_score='raise')

grid_result = grid_search.fit(Xt, Y)

print(grid_result.best_params_)


gs_time = time() - tic
print('Tiempo en segundos: ',gs_time)
tiempoTotal=GetTime(gs_time)
print('Tiempo: ',tiempoTotal)

gs1=grid_result